In [ ]:
!pip install -q openai=="1.30.3" requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
from google.colab import userdata
import requests
import json

In [ ]:
client = OpenAI(api_key=userdata.get('openai_api'))

In [ ]:
weather_api_key = userdata.get('opeanweather')

In [ ]:
def get_weather(city: str):
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}&units=metric'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        weather = {
            "city": city,
            "temperature": data['main']['temp'],
            "description": data['weather'][0]['description'],
            "humidity": data['main']['humidity'],
            "pressure": data['main']['pressure']
        }
        return weather
    else:
        return None

In [ ]:
get_weather("jaffna")

{'city': 'jaffna',
 'temperature': 29.22,
 'description': 'broken clouds',
 'humidity': 76,
 'pressure': 1006}

In [ ]:
weather_function = {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },

                },
                "required": ["location"],
            },
        }
    }

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."}
]

while True:
    user_input = str(input("Ask your question or If you want to stop it type as quit:- "))

    if user_input=='quit':
      break

    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=[weather_function],
        tool_choice="auto",
    )

    tool_calls = response.choices[0].message.tool_calls

    if tool_calls:
        available_functions = {
            "get_weather": get_weather,
        }

        for tool_call in tool_calls:
            print(f"Function using: {tool_call.function.name}")
            # print(f"Params:{tool_call.function.arguments}")

            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                city=function_args.get("location"),
            )
            # print(f"API: {function_response}")

            messages.append(
                {
                    "role": "function",
                    "name": function_name,
                    "content": json.dumps(function_response),
                }
            )

            second_response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
            )

            messages.append({
                "role": "assistant",
                "content": second_response.choices[0].message.content
            })

            print(second_response.choices[0].message.content)

    else:
        messages.append({
                "role": "assistant",
                "content": response.choices[0].message.content
            })

        print(response.choices[0].message.content)
    print("")

Ask your question or If you want to stop it type as quit:- what is the weather in jaffna
Function using: get_weather
The current weather in Jaffna is 29.22°C with broken clouds. The humidity is at 76% and the atmospheric pressure is 1006 hPa.

Ask your question or If you want to stop it type as quit:- where is jaffna located
Jaffna is located in the northern part of Sri Lanka. It is a city on the northern tip of the Jaffna Peninsula.

Ask your question or If you want to stop it type as quit:- q
If you have any more questions or need further assistance, feel free to ask!

Ask your question or If you want to stop it type as quit:- quit
